# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [70]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [71]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [72]:
# Import the necessary CSVs to Pandas DataFrames
# Read the census data into a Pandas DataFrame
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
file_path = Path("Data/neighborhoods_coordinates.csv")
sfo_data_loc = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [73]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    sfo_housing_average = sfo_data.groupby('year').mean()
    sfo_housing_stat = sfo_housing_average['housing_units'].describe()
    housing_units_per_year = px.bar(sfo_housing_average, y='housing_units',
                                    title='Housing Units in San Francisco from 2010 to 2016')
    housing_units_per_year.update_yaxes(range=[350000, 400000])
    return housing_units_per_year



def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    sfo_rent = sfo_data[['gross_rent']].groupby('year').mean()
    average_gross_rent = px.line(sfo_rent, y='gross_rent',
                         title='Gross Rent in San Francisco from 2010 to 2016')
    return average_gross_rent



def average_sales_price():
    """Average Sales Price Per Year."""
    sfo_sales = sfo_data[['sale_price_sqr_foot']].groupby('year').mean()
    average_sales_price = px.line(sfo_sales, y='sale_price_sqr_foot',
                         title='Sale Price per Sqft in San Francisco from 2010 to 2016')
    return average_sales_price
    



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    sfo_housing_average_neighborhood = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
    return sfo_housing_average_neighborhood.hvplot(x='year', 
                                                   y=['gross_rent','sale_price_sqr_foot'],
                                                   kind = 'bar', 
                                                   groupby='neighborhood', 
                                                   rot=90, 
                                                   height=500, 
                                                   xlabel='Year', 
                                                   ylabel='Dollar Amount')




def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    top_10_neighborhoods_raw = sfo_data.groupby(['neighborhood']).mean().reset_index()
    top_10_neighborhoods_sorted = top_10_neighborhoods_raw.sort_values(by ='sale_price_sqr_foot', 
                                                                       ascending=False)
    return px.bar(top_10_neighborhoods_sorted[0:9],
                                              x='neighborhood', 
                                              y='sale_price_sqr_foot', 
                                              title='Sale Price per Square Foot for Top Ten Neighborhoods')
   


def most_expensive_neighborhoods_rent_sales():
    top_10_neighborhoods_raw = sfo_data.groupby(['neighborhood']).mean().reset_index()
    top_10_neighborhoods_sorted = top_10_neighborhoods_raw.sort_values(by ='gross_rent', 
                                                                       ascending=False)
    return px.bar(top_10_neighborhoods_sorted[0:9],x='neighborhood', 
                                              y=['sale_price_sqr_foot','gross_rent'], 
                                              title='Gross Rent for Top Ten Neighborhoods')   


    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    sfo_housing_average_neighborhood = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
    top_10_neighborhoods_raw = sfo_data.groupby(['neighborhood']).mean().reset_index()
    top_10_neighborhoods_sorted = top_10_neighborhoods_raw.sort_values(by ='sale_price_sqr_foot', ascending=False)
    df_expensive_neighborhoods = top_10_neighborhoods_sorted[0:9]
    return px.parallel_coordinates(df_expensive_neighborhoods, 
                                   color='sale_price_sqr_foot')


def parallel_categories():
    """Parallel Categories Plot."""
    sfo_housing_average_neighborhood = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
    top_10_neighborhoods_raw = sfo_data.groupby(['neighborhood']).mean().reset_index()
    top_10_neighborhoods_sorted = top_10_neighborhoods_raw.sort_values(by ='sale_price_sqr_foot', 
                                                                       ascending=False)
    df_expensive_neighborhoods = top_10_neighborhoods_sorted[0:9]
    return px.parallel_categories(df_expensive_neighborhoods,dimensions=["neighborhood", 
                                                                  "sale_price_sqr_foot", 
                                                                  "housing_units", "gross_rent"],
                           color="sale_price_sqr_foot",
                           color_continuous_scale=px.colors.sequential.Inferno)



def neighborhood_map():
    """Neighborhood Map."""
    calc_averages_neighborhood = sfo_data.groupby('neighborhood').mean().reset_index()
    final = pd.concat([sfo_data_loc, calc_averages_neighborhood], axis="columns", join="inner")
    final = final.loc[:,~final.columns.duplicated()]
    return px.scatter_mapbox(
    final,
    lat="Lat",
    lon="Lon",
    size="gross_rent",
    color="neighborhood")



def sunburst():
    sfo_housing_average_neighborhood = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
    top_10_neighborhoods_raw = sfo_data.groupby(['neighborhood']).mean().reset_index()
    top_10_neighborhoods_sorted = top_10_neighborhoods_raw.sort_values(by ='sale_price_sqr_foot', ascending=False)
    df_expensive_neighborhoods = top_10_neighborhoods_sorted[0:9]
    sfo_housing_average_neighborhood = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
    df_expensive_neighborhoods_per_year = sfo_housing_average_neighborhood[sfo_housing_average_neighborhood["neighborhood"].isin(df_expensive_neighborhoods["neighborhood"])]
    return px.sunburst(df_expensive_neighborhoods_per_year, 
                  path=['year','neighborhood'], 
                  values='sale_price_sqr_foot',
                  color='gross_rent', 
                  color_continuous_scale='Blues',
                  width=750,
                  height=750)
    
  


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [74]:
# Create a Title for the Dashboard
title = pn.Row(
    '# Summary Statistics in Housing Costs, by Neighborhoods, in Sanfrancisco',
    '## 2010 to 2016')

# Create a tab layout for the dashboard
tab1 = pn.Tabs(
    ("Total Housing Units", housing_units_per_year()),
    ("Average Gross Rent",average_gross_rent()),
    ("Average Sales Price", average_sales_price()),
    ("Average Sales Price and Rent By Neighborhood",average_price_by_neighborhood()))
tab2 = pn.Tabs(
    ("Top Ten Most Expensive Neighborhoods By Sales",top_most_expensive_neighborhoods()),
    ("Top Ten Most Expensive Neighborhoods By Sales and Gross Rent",(most_expensive_neighborhoods_rent_sales())),
    ("Parallel Categories of Top Ten Neighborhoods",(parallel_categories())),
    ("Parallel Coordinates",(parallel_coordinates())))
tab3 = pn.Tabs(("Sunburst Chart of Sale Price",(sunburst())),
              ("Neighborhood Map",(neighborhood_map()))
              )

# Create the dashboard
dashboard = pn.Column(title,tab1,tab2,tab3)

## Serve the Panel Dashboard

In [75]:
dashboard

Column
    [0] Row
        [0] Markdown(str)
        [1] Markdown(str)
    [1] Tabs
        [0] Plotly(Figure, name='Total Housing Units')
        [1] Plotly(Figure, name='Average Gross Rent')
        [2] Plotly(Figure, name='Average Sales Price')
        [3] Row(name='Average Sales P...)
            [0] HoloViews(DynamicMap, name='Average Sales P...)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [2] Tabs
        [0] Plotly(Figure, name='Top Ten Most E...)
        [1] Plotly(Figure, name='Top Ten Most E...)
        [2] Plotly(Figure, name='Parallel Categories o...)
        [3] Plotly(Figure, name='Parallel Coordinates')
    [3] Tabs
        [0] Plotly(Figure, name='Sunburst Chart o...)
        [1] Plotly(Figure, name='Neighborhood Map')